### This notebook is for finding a bound in the case where our category is of prime rank (Note: this notebook uses Julia as its kernel)

Below is an edited version of Emily's code for the implementation of Algorithm 4.1 in Rowell-Bruillard paper

In [1]:
# first 9 terms of sylvesters sequence
Sylvester_seq = [1, 2, 3, 7, 43, 1807, 3263443, 10650056950807, 113423713055421844361000443] 

The cell below creates a library of potential solutions

In [2]:
#Follows algorithm 4.1 in http://arxiv.org/abs/1012.0814v2

def generate_S_lib(rank_of_C, bounding_sequence):
    #step 1 - generate the set S_2
    n = rank_of_C
    u_seq = bounding_sequence
    S_2 = []

    for j in range(2, n+1):
        upp = math.sqrt(u_seq[n-1]/j) #upper bound on i where pairs in S_2 are (j, j*i^2)
        for i in range(1, math.floor(upp) + 1):
            S_2.append((j, j * i * i))

    S_library = {}
    S_library[2] = list(S_2)
    #step 2 - generate the set S_k+1 for k <= n-2 
    for k in range(3, n+1):

        S_library[k] = []
        # condition a
        for j_list in S_library[k-1]:
            partial = sum([(1/x_i) for x_i in j_list])
            #condition b
            bound_1 = max(j_list[-2], k)
            bound_2 = min(j_list[-1], u_seq[k - 1]*(n - k + 1))
            for J in range(bound_1, bound_2+1):

                #condition c and d
                if math.floor(math.sqrt(j_list[-1]/J)) == math.sqrt(j_list[-1]/J) and (1/J) + partial <= 1:

                    temp = j_list[-1]
                    newlist = list(j_list)
                    newlist.remove(newlist[-1])
                    newlist.append(J)
                    newlist.append(temp)
                    S_library[k].append(newlist)
                    
    return S_library

This cell returns the lists of solutions x_i in a library that fit the requirement that they sum to 1

In [3]:
def find_x_i(rank_of_C, Sylvester_sequence): 
    n = rank_of_C
    u_seq = Sylvester_sequence
    S_library = generate_S_lib(n, u_seq)
    solutions = []               
    for possible_sol in S_library[n]:
        if 1 - sum([(1/x_i) for x_i in possible_sol]) <= 1/u_seq[n-1]: 
            #this should fix any rounding error weirdness
            solutions.append(list(possible_sol))

    return solutions

### The cell below contains code for the case of categories with prime rank


is_solution takes in a list and determines whether the list is a valid solution, can be completed to a valid solution, or cannot be completed to a valid solution

find_solutions returns a list of possible x_1, x_2,...,x_n-1

find_possible_dim_C returns the list of the dimensions of C corresponding to the solutions found by "find_solutions"

find_max_power returns the same list as in find_possible_dim_C but takes the values log base p

In [4]:
from sage.functions.log import logb

# set prime factors in question
p = 3
MAX_LENGTH = 10
# Note that MAX_LENGTH:= rank - 1
TERMINAL_POWER = 5

possible_denoms = []

for i in range(0,TERMINAL_POWER + 1):
    possible_denoms.append(p^i)
        
possible_denoms.sort()
possible_denoms.remove(1)

nice_sums = [(n-1)/n for n in possible_denoms]

from enum import Enum
Status = Enum('Solution_Status', ['INVALID', 'VALID', 'INCOMPLETE'])


def is_solution(candidate_sol):
    if len(candidate_sol) > MAX_LENGTH: 
        return Status.INVALID
    elif sum([(1/x_i) for x_i in candidate_sol]) > 1:
        return Status.INVALID
    # condition to return true (need to add to solutions list)
    elif len(candidate_sol) == MAX_LENGTH and sum([(1/x_i) for x_i in candidate_sol]) in nice_sums:
        return Status.VALID
    else:
        return Status.INCOMPLETE

def __find_solutions_loop(partial_sol, solutions_list):
    match is_solution(partial_sol):
        case Status.INVALID:
            return
        case Status.VALID:
            solutions_list.append(list(partial_sol))
        case Status.INCOMPLETE:
            # loop through denominators, check if its bigger (or equal)
            for i in possible_denoms: 
            
                if len(partial_sol) == 0 or i >= partial_sol[-1]:
                    #add to list, recursively call function
                    partial_sol.append(i)
                    __find_solutions_loop(partial_sol, solutions_list)
                
                    #remove from list
                    partial_sol.remove(i)

    return solutions_list

def find_solutions(init_partial_sol = []):
    return __find_solutions_loop(init_partial_sol, [])
                    


#print(solutions)



def find_possible_dim_C():
    denoms = []
    for solution in find_solutions([]):
        x = 0 
        for i in solution:
            x=x+1/i
        denoms.append(x.denominator())
    return denoms

def find_max_power(prime):
    denoms = []
    for solution in find_solutions([]):
        x = 0 
        for i in solution:
            x=x+1/i
        denoms.append(logb(x.denominator(), prime))
        
    denoms.sort()
    return denoms
        

full_solutions= find_solutions()
    

for sol in full_solutions:
    s = 0
    for i in sol:
        s += 1/i
    sol.append(s.denominator())
    


[[3, 3, 9, 9, 27, 27, 81, 81, 243, 243, 243], [3, 3, 9, 9, 27, 27, 81, 243, 243, 243, 81], [3, 3, 9, 9, 27, 81, 81, 81, 81, 81, 81], [3, 3, 9, 9, 27, 81, 81, 243, 243, 243, 27], [3, 3, 9, 9, 81, 81, 81, 81, 81, 81, 27], [3, 3, 9, 27, 27, 27, 27, 27, 81, 81, 81], [3, 3, 9, 27, 27, 27, 27, 81, 81, 81, 27], [3, 3, 9, 27, 27, 81, 81, 243, 243, 243, 9], [3, 3, 9, 27, 81, 81, 81, 81, 81, 81, 9], [3, 3, 27, 27, 27, 27, 27, 27, 27, 27, 27], [3, 3, 27, 27, 27, 27, 27, 81, 81, 81, 9], [3, 9, 9, 9, 9, 9, 27, 27, 81, 81, 81], [3, 9, 9, 9, 9, 9, 27, 81, 81, 81, 27], [3, 9, 9, 9, 9, 27, 27, 27, 27, 27, 27], [3, 9, 9, 9, 9, 27, 27, 81, 81, 81, 9], [3, 9, 9, 9, 27, 27, 27, 27, 27, 27, 9], [3, 9, 9, 27, 27, 81, 81, 243, 243, 243, 3], [3, 9, 9, 27, 81, 81, 81, 81, 81, 81, 3], [3, 9, 27, 27, 27, 27, 27, 81, 81, 81, 3], [3, 27, 27, 27, 27, 27, 27, 27, 27, 27, 3], [9, 9, 9, 9, 9, 9, 9, 9, 27, 27, 27], [9, 9, 9, 9, 9, 9, 9, 27, 27, 27, 9], [9, 9, 9, 9, 9, 27, 27, 81, 81, 81, 3], [9, 9, 9, 9, 27, 27, 27, 27,

The code in the cell below counts the number of distinct solutions given a fixed prime p and rank R.

In [14]:
solutions_dictionary =[]

for solution in full_solutions:
    
    sol_dict = {}
    
    for i in solution:
        
        if i in sol_dict.keys():
            sol_dict[i] += 1
        else:
            sol_dict[i] = 1

    solutions_dictionary.append(sol_dict)
    


dup_free = [solutions_dictionary[0]]

for i in range(len(solutions_dictionary)):
        unique = True
        for item in dup_free:
            
            if solutions_dictionary[i] == item:
                
                unique = False
        if unique:
            
            dup_free.append(solutions_dictionary[i])
            
print(dup_free)
print(len(dup_free))

[{3: 2, 9: 2, 27: 2, 81: 2, 243: 3}, {3: 2, 9: 2, 27: 1, 81: 6}, {3: 2, 9: 1, 27: 5, 81: 3}, {3: 2, 27: 9}, {3: 1, 9: 5, 27: 2, 81: 3}, {3: 1, 9: 4, 27: 6}, {9: 8, 27: 3}]
7
